In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
from eth_defi.token import TokenDetails, fetch_erc20_details

In [9]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    args  = Conversion().decode_data(evt["data"])
    token0_address = Conversion().convert_uint256_hex_string_to_address(topics[1])
    token1_address = Conversion().convert_uint256_hex_string_to_address(topics[2])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address    
    if(len(args) == 5):
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[0], signed=direction(args[0])) 
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[1], signed=direction(args[1])) 
        event['details']['sqrt_price_x96'] = Conversion().convert_int256_bytes_to_int(args[2]) 
        event['details']['liquidity'] = Conversion().convert_int256_bytes_to_int(args[3]) 
        event['details']['tick'] = Conversion().convert_int256_bytes_to_int(args[4], signed=direction(args[4])) 
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,356,527 tx:0x2db21c556098f14a3a06b82dda9cd5d35a5259fe93fb2adb9e0857d80e039af5
Swap at block:60,356,527 tx:0xe3aaa697b9bb94df8f693046e51394fc2013e9d8607398ac32e27cce9c442299
Swap at block:60,356,527 tx:0x76612fc75474463d8c9c889d65c53f14932a6b145c0ae37641078df16d54f7bc
Swap at block:60,356,527 tx:0x76612fc75474463d8c9c889d65c53f14932a6b145c0ae37641078df16d54f7bc
Swap at block:60,356,527 tx:0x76612fc75474463d8c9c889d65c53f14932a6b145c0ae37641078df16d54f7bc
Swap at block:60,356,527 tx:0x49b6d25966a99d0fe52f9cc0adeacde5a0fa772ae827c35755b5769ab7ffb543
Swap at block:60,356,527 tx:0xcb64c1db35da7b779f2b1ec38f259573727b81266dc9045251e08006344a072a
Swap at block:60,356,527 tx:0xcc3eb110608f18c50f6174b5d92d34968866cf54e62756d4eac87e0887e8cdb8
Swap at block:60,356,527 tx:0xcc3eb110608f18c50f6174b5d92d34968866cf54e62756d4eac87e0887e8cdb8
Swap at block:60,356,527 tx:0xe4afb192d29156b170a7ff56a5ff5a01baf2371a93a54bc87e93b4d09f16daba
Swap at block:60,356,527 tx:0xe4afb192d29156b170a7

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,swap,uniswap_v3,0xa4d8c89f0c20efbe54cba9e7e7a7e509056228d9,0x2db21c556098f14a3a06b82dda9cd5d35a5259fe93fb...,60356527,1723156540,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv3pool,swap,uniswap_v3,0xbb98b3d2b18aef63a3178023a920971cf5f29be4,0xe3aaa697b9bb94df8f693046e51394fc2013e9d86073...,60356527,1723156540,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv3pool,swap,uniswap_v3,0xd457b0feb8fa6fada96e1e0274110c5734d61bc7,0x76612fc75474463d8c9c889d65c53f14932a6b145c0a...,60356527,1723156540,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv3pool,swap,uniswap_v3,0x327a4202fb79eac04a4e974d9fef61c2e2e0e33d,0x76612fc75474463d8c9c889d65c53f14932a6b145c0a...,60356527,1723156540,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv3pool,swap,uniswap_v3,0xbb98b3d2b18aef63a3178023a920971cf5f29be4,0x76612fc75474463d8c9c889d65c53f14932a6b145c0a...,60356527,1723156540,{'web3_type': <class 'web3._utils.datatypes.Sw...
...,...,...,...,...,...,...,...,...,...
109,polygon,uniswapv3pool,swap,uniswap_v3,0x55caabb0d2b704fd0ef8192a7e35d8837e678207,0xa244dab4aa2ec517f179f04b0e235ba73af9f28ae213...,60356535,1723156558,{'web3_type': <class 'web3._utils.datatypes.Sw...
110,polygon,uniswapv3pool,swap,uniswap_v3,0x44759c34a5c31a69d63248ecbaf5292574d26bf2,0x0236d5b044cabb67e208dbf43ea86b3e2643225dee4e...,60356535,1723156558,{'web3_type': <class 'web3._utils.datatypes.Sw...
111,polygon,uniswapv3pool,swap,uniswap_v3,0x45dda9cb7c25131df268515131f647d726f50608,0x09184f851a91990bc3ef6411a20f973799a959dad54e...,60356535,1723156558,{'web3_type': <class 'web3._utils.datatypes.Sw...
112,polygon,uniswapv3pool,swap,uniswap_v3,0x9ceff2f5138fc59eb925d270b8a7a9c02a1810f2,0x312453bcd82840b301e5888bd3fcae2481b56a9bf40e...,60356535,1723156558,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [11]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xa4d8c89f0c20efbe54cba9e7e7a7e509056228d9',
  'tx_hash': '0x2db21c556098f14a3a06b82dda9cd5d35a5259fe93fb2adb9e0857d80e039af5',
  'blk_num': 60356527,
  'timestamp': 1723156540,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x802b65b5d9016621E66003aeD0b16615093f328b',
   'token1': '0x802b65b5d9016621E66003aeD0b16615093f328b',
   'token0_symbol': None,
   'token1_symbol': None,
   'amount0': -682091476,
   'amount1': 257304911874366860,
   'sqrt_price_x96': 1538563346934715238462476008119949,
   'liquidity': 69638190308949303,
   'tick': 197490}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xbb98b3d2b18aef63a3178023a920971cf5f29be4',
  'tx_hash': '0xe3aaa697b9bb94df8f693046e51394fc2013e9d8607398ac32e27cce9c442299',
  'blk_num': 60356527,
  'timestamp': 1723156540,
  'details': {